### 1. Importar Lista de vehículos ligeros eléctricos con precios en Europa "WEB SCRAPING":

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL base de la página a hacer scraping
base_url = 'https://ev-database.org/'
url = base_url

# Lista para almacenar los datos
data = []

# Diccionario para almacenar los precios por vehículo y país
prices = {}

# Realizar solicitudes GET a cada página
while True:
    # Realizar la solicitud GET a la página
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Parsear el contenido HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos los elementos <div> con la clase 'list-item'
        list_items = soup.find_all('div', class_='list-item')

        # Iterar sobre los elementos 'list-item' y extraer información
        for item in list_items:
            # Buscar el elemento <div> con la clase 'data-wrapper' dentro del elemento 'list-item'
            data_wrapper = item.find('div', class_='data-wrapper')
            if data_wrapper:
                # Buscar el elemento <a> con la clase 'title'
                title_link = data_wrapper.find('a', class_='title')
                if title_link:
                    # Obtener el nombre del vehículo
                    car_name = title_link.text.strip()

                    # Buscar el elemento <div> con la clase 'specs' para obtener información adicional
                    specs_div = item.find('div', class_='specs')
                    # Velocidad máxima
                    top_speed_tag = specs_div.find('span', class_='topspeed')
                    top_speed = top_speed_tag.text.strip() if top_speed_tag else "Velocidad máxima no encontrada"
                    # Tiempo de 0 a 100 km/h
                    acceleration_tag = specs_div.find('span', class_='acceleration')
                    acceleration = acceleration_tag.text.strip() if acceleration_tag else "Tiempo de 0 a 100 km/h no encontrado"
                    # Rango
                    range_tag = specs_div.find('span', class_='erange_real')
                    range_value = range_tag.text.strip() if range_tag else "Rango no encontrado"
                    # Carga rápida
                    fastcharge_speed_tag = specs_div.find('span', class_='fastcharge_speed_print')
                    fastcharge_speed = fastcharge_speed_tag.text.strip() if fastcharge_speed_tag else "Velocidad de carga rápida no encontrada"

                    # Encontrar todos los elementos <span> con la clase 'price_buy' dentro del elemento de precios
                    pricing_elements = item.find_all('span', class_='price_buy')
                    # Iterar sobre los elementos de precio
                    for pricing_element in pricing_elements:
                        # Extraer el texto dentro del elemento como el precio
                        price_text = pricing_element.text.strip()
                        # Extraer el país de la clase del elemento (por ejemplo, 'country_de', 'country_nl', etc.)
                        country_code = pricing_element.find('span', class_='flag-icon').attrs['class'][1].replace('country_', '').upper()
                        # Agregar el precio al diccionario de precios, utilizando el nombre del vehículo como clave
                        if car_name not in prices:
                            prices[car_name] = {}
                        prices[car_name][country_code] = price_text

                    # Agregar los datos a la lista
                    data.append({
                        "Nombre del vehículo": car_name,
                        "Velocidad máxima": top_speed,
                        "Tiempo de 0 a 100 km/h": acceleration,
                        "Rango": range_value,
                        "Velocidad de carga rápida": fastcharge_speed,
                        **prices[car_name]  # Agregar precios al diccionario de datos
                    })

        # Encontrar el enlace a la siguiente página, si existe
        next_page_link = soup.find('a', class_='next')
        if next_page_link:
            url = base_url + next_page_link['href']
        else:
            break  # Salir del bucle si no hay más páginas disponibles
    else:
        print('Error al acceder a la página:', response.status_code)
        break


### 2. Corregir formato de números y renombrar columnas:

In [ ]:
# Crear un DataFrame de Pandas con los datos
df = pd.DataFrame(data)

# Renombrar las columnas
df.rename(columns={
    'Velocidad máxima': 'Velocidad Máxima (Km/h)',
    'Tiempo de 0 a 100 km/h': 'Aceleración 0 a 100 Km/h (seg)',
    'Rango': 'Rango (Km)',
    'Velocidad de carga rápida': 'Carga rápida (Km/h)'
}, inplace=True)

# Limpiar los datos numéricos
df['Velocidad Máxima (Km/h)'] = df['Velocidad Máxima (Km/h)'].str.extract('(\d+)').astype(float)
df['Aceleración 0 a 100 Km/h (seg)'] = df['Aceleración 0 a 100 Km/h (seg)'].str.extract('(\d+\.?\d*)').astype(float)
df['Rango (Km)'] = df['Rango (Km)'].str.extract('(\d+)').astype(float)
df['Carga rápida (Km/h)'] = df['Carga rápida (Km/h)'].str.extract('(\d+)').astype(float)


### 3. Renombrar columnas de precios y arreglar valores:

In [61]:
# Renombrar las columnas de precio
df.rename(columns={
    'FLAG-ICON-DE': 'Alemania (€)',
    'FLAG-ICON-NL': 'Holanda (€)',
    'FLAG-ICON-GB': 'Inglaterra (€)'
}, inplace=True)

# Limpiar los valores de las columnas de precio
for country in ['Alemania (€)', 'Holanda (€)', 'Inglaterra (€)']:
    df[country] = df[country].str.replace('*', '')  # Eliminar asteriscos
    df[country] = df[country].str.replace('[^\d,.]', '', regex=True)  # Eliminar caracteres no numéricos
    df[country] = df[country].str.replace(',', '', regex=False)  # Eliminar comas como separadores de miles
    df[country] = pd.to_numeric(df[country], errors='coerce')  # Convertir a tipo numérico

### 4. Unificar precios a promedio Europa en Euros:

In [64]:
# Crear la columna "Europa (€)" que promedia los valores de las tres columnas de precios, ignorando los valores nulos
df['Europa (€)'] = df[['Alemania (€)', 'Holanda (€)', 'Inglaterra (€)']].mean(axis=1, skipna=True)

# Eliminar las columnas individuales de precios
df.drop(columns=['Alemania (€)', 'Holanda (€)', 'Inglaterra (€)'], inplace=True)


### 5. Eliminar registros nulos y duplicados:

In [66]:
# Eliminar registros nulos
df.dropna(inplace=True)

# Eliminar registros duplicados
df.drop_duplicates(inplace=True)

In [68]:
# Guardar el DataFrame en un archivo CSV
#df.to_csv('../DataClean/datos_vehiculos.csv', index=False)